# Progetto DSIM - Demo Live Audio

---

Autori: Confalonieri Riccardo | Mariani Ginevra | Mora Lorenzo <br>
E-mail: r.confalonieri5@campus.unimib.it | g.mariani34@campus.unimib.it |  l.mora4@campus.unimib.it

Lo scopo di questo notebook è implementare una live demo per dimostrare l'efficacia dei modelli implementati per il task di speech recognition. Siccome la demo necessita del microfono (e della telecamera anche se non essenziale) per acquisire gli audio è necessario eseguire il file localmente e non in Colab! Inoltre è importante controllare il path da cui si caricano i modelli trainati per evitare errori. 

## Import packages

In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
from playsound import playsound
import soundfile as sf
import numpy as np
import os
import librosa
import tensorflow.keras as keras
from keras.models import load_model
import cv2 as cv
import pickle
from scipy.io import wavfile as wav
import time
import sklearn
from sklearn.model_selection import GridSearchCV

## Extract features
In questa sezione vengono riportati tutti i metodi necessari per estrarre le features di riferimento in base al modello utilizzato per la predizione.

In [ ]:
def check_length(input, size):
    # Elimina eventuali valori oltre il numero consentito
    output = input[0:min(size, input.shape[0])]
    # Aggiungi valori nulli per raggiungere la dimensione richiesta
    output = np.concatenate((output, np.zeros(size-output.shape[0])))
    return output

In [ ]:
# ritorna mfcc dell'audio
def mfcc(input, rate = 44100):
    return librosa.feature.mfcc(input, sr = rate, n_mfcc = 13)

# crea lo spettrogramma di un audio 
def get_spectrogram(path, output_shape = (64, 64)):
    data, samplerate = sf.read(path)
    # data = check_length(data, multi = True)[2]
    # Spettrogramma
    audio_stft = librosa.amplitude_to_db(np.abs(librosa.stft(data.astype(float))))
    scaled_stft = audio_stft + abs(np.min(audio_stft))

    image = scaled_stft/np.max(scaled_stft)*255
    # Espansione dell'immagine a tre dimensioni
    image = np.repeat(np.expand_dims(image, 2), 3, 2).astype('uint8')
    # Resizing
    image = cv.resize(image, output_shape)

    return image

## Demo
Questa sezione riporta le funzioni necessarie per implementare la demo correttamente.

In [ ]:
def predictVoice(audio_path, size, model):
    '''
      Determina lo speaker dell'audio in input.

      :param audio_path = percorso per audio da classificare
      :param size = dimensione default dell'audio in secondi
      :param model = modello per la classificazione
    '''

    label_authors = {0: 'Ginevra', 1:'Lorenzo', 2:'Riccardo'}
      
    author = -1 # nel caso di errore
    sr, audio = wav.read(audio_path)
    # verifico che l'audio sia della lunghezza giusta, nel caso lo correggo
    audio = check_length(audio, size*sr)
        
    
    if isinstance(model, sklearn.model_selection._search.GridSearchCV):
        features = mfcc(audio).flatten()
        author = model.predict(features.reshape(-1, features.shape[0]))[0]
        
    
    elif (isinstance(model, keras.Model) and 
        list(model.input.shape) == [None, 64, 64, 3]):
        img = get_spectrogram(audio_path)
        image = np.expand_dims(img, axis = 0)

        author = model.predict(image)
        # estraggo il singolo autore
        author = np.argmax(author, axis = 1)
        # converto da numero a stringa di riferimento
        author = label_authors[author[0]]
        
    elif isinstance(model, keras.Model):
        # leggo l'audio dal file
        _, audio = wav.read(audio_path)
            
        # estraggo le features con mfcc
        feats = mfcc(audio)
        
        # adatto le features alla shape richiesta dal modello
        feats = np.array(feats).reshape(-1,feats.shape[0],feats.shape[1],1) 
        # effettuo la predizione
        author = model.predict(feats)
        # estraggo il singolo autore
        author = np.argmax(author, axis = 1)
        # converto da numero a stringa di riferimento
        author = label_authors[author[0]]
        
        
    return author

In [ ]:
# funzione che salva il file su disco
def save_file_helper(audio, rec_rate):
    # Salvataggio registrazione voce
    t = time.localtime()
    timestamp = time.strftime('%m-%d-%Y_%H%M%S', t)
    nome_file = 'rec_' + timestamp + '.wav'
    write(nome_file, rate = rec_rate, data = audio)
    
    return nome_file


def demo(model, duration = 5, rec_rate = 44100, archive = False):
    '''
        :param model = classificatore per riconoscimento vocale
        :param duration = durata dei file audio in secondi
        :param rec_rate = sample rate dei file audio
        :param archive = True per memorizzare file
        
    '''
    comando = "Premere 'spacebar' per registrare o 'q' per fermare la registrazione."
    registra = False
    wait_time = time.time()
    
    try:
        videoCap = cv.VideoCapture(0)

        if (videoCap.isOpened() == False):
            print("Errore apertura video camera o file.")

        while(videoCap.isOpened()):
            frameOk, frame = videoCap.read()

            if not frameOk:
                print('Impossibile riconoscere l\'immagine')
                # Break the loop
                break

            # se l'utente ha cliccato la barra e sono passati 0.3 secondi inizio a registrare
            # gli 0.3 sec non sono necessari sono solo un aiuto per iniziare a parlare e registrare
            # un audio di solo parole effettive
            if registra and (time.time() - wait_time > 0.3):
                # registro l'audio della lunghezza necessaria
                audio = sd.rec(int(duration * rec_rate), 
                               samplerate = rec_rate, channels = 1, blocking=True)


                # salvo l'audio su disco per processarlo correttamente
                tmp_filename = save_file_helper(audio, rec_rate)

                # Identificazione speaker attraverso modello
                author = predictVoice(tmp_filename, 2, model)

                # aggiorno il comando
                if author == 'Ginevra':
                    comando = "Beccata " + author + "! Come stai?"
                else:
                    comando = "Beccato " + author + "! Come stai?"


                # risetto a false la variabile registra
                registra = False

                # se non richiesto cancello il file da disco
                if archive == False:
                    os.remove(tmp_filename)

                wait_time = time.time()

            # verifico se ha premuto lo spazio, nel caso cambio comando da visualizzare
            # e mi preparo per registrare l'audio
            if cv.waitKey(25) & 0xFF == ord(' '):
                comando = 'Sto registrando l\'audio...'
                wait_time = time.time()
                registra = True


            # Premere 'q' per uscire    
            if cv.waitKey(10) & 0xFF == ord('q'):
                break

            # risetto il comando iniziale dopo che ho predetto l'utente
            if (registra == False) and (time.time() - wait_time > 2):
                comando = "Premere 'spacebar' per registrare o 'q' per fermare la registrazione."

            # cambio comando e visualizzazione su video
            cv.putText(frame, comando, (15, 37), 
                       cv.FONT_HERSHEY_SIMPLEX, 0.75, (20,38,65), 2)
            # visualizzo la finestra
            cv.imshow('Riconoscimento identita\' dello speaker.', frame)


        # Rilascia la video camera quando tutto è eseguito
        videoCap.release()
        cv.destroyAllWindows()
    except:
        # Rilascia la video camera quando tutto è eseguito
        videoCap.release()
        cv.destroyAllWindows()

In [ ]:
model = load_model('./scratch.h5')
# model = load_model('./modelSpectrogram_compressed.h5')
# model = pickle.load(open('./svm_augm.pkl', 'rb'))
demo(model, 2, 44100, False)

In [2]:
# download notebook in .html extension
%%shell
jupyter nbconvert --to html '/content/drive/MyDrive/Confalonieri_Mariani_Mora_DSIM/Notebook/Processing-1D/3_DemoLive.ipynb'

[NbConvertApp] Converting notebook /content/drive/MyDrive/Confalonieri_Mariani_Mora_DSIM/Notebook/Processing-1D/3_DemoLive.ipynb to html
[NbConvertApp] Writing 305701 bytes to /content/drive/MyDrive/Confalonieri_Mariani_Mora_DSIM/Notebook/Processing-1D/3_DemoLive.html
